In [1]:
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import scipy
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
import random
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
print (scipy.__version__)
from IPython.display import Image
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from flask import Flask, request, jsonify, render_template
from scipy.sparse import csr_matrix

1.9.3


In [2]:
data = pd.read_csv('books_data/books.csv', sep=";", 
                    error_bad_lines=False, encoding="latin-1")

C:\Users\giaco\AppData\Local\Temp\ipykernel_39788\1051877748.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('books_data/books.csv', sep=";",
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 f

In [3]:
# Drop or fill missing values in relevant columns
data = data.dropna(subset=['Book-Title', 'Book-Author', 'Publisher'])

In [4]:
data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
data_qualitative = data[["Book-Title","Book-Author","Publisher"]]
data_qualitative.head()

,Book-Title,Book-Author,Publisher
0,Classical Mythology,Mark P. O. Morford,Oxford University Press
1,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada
2,Decision in Normandy,Carlo D'Este,HarperPerennial
3,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux
4,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company


***AGGIUNGERE GENRE DEFINITION IF POSSIBLE***

In [6]:
# Randomly sample a subset of the data
sampled_data = data_qualitative.sample(frac=0.25, random_state=42)

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
qualitative_data = sampled_data.copy()

# Function to recommend books based on content similarity with unique book names
def get_unique_recommendations(book_title, author, num_recommendations=10):
    global qualitative_data  # Declare qualitative_data as a global variable

    if book_title not in qualitative_data['Book-Title'].values:
        print(f"Warning: Book with title '{book_title}' not found in the dataset. Recommendations are based on available data.")
        # Add the new book to the dataset
        new_data = {'Book-Title': [book_title], 'Book-Author': [author]}
        qualitative_data = qualitative_data.append(pd.DataFrame(new_data), ignore_index=True)

    # Combine text features into a single column
    qualitative_data['TextFeatures'] = qualitative_data['Book-Title'] + ' ' + qualitative_data['Book-Author']

    # Create a TF-IDF vectorizer for text features (Author and Genre)
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # Apply TF-IDF vectorization to text features
    tfidf_matrix = tfidf_vectorizer.fit_transform(qualitative_data['TextFeatures'])

    # Convert tfidf_matrix to a sparse matrix
    # tfidf_sparse = csr_matrix(tfidf_matrix)

    # Calculate cosine similarities between books based on text features
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    # If the title is in the dataset, proceed with the usual calculation
    if book_title in qualitative_data['Book-Title'].values:
        idx = qualitative_data[qualitative_data['Book-Title'] == book_title].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Sort and get the top 100 most similar books (excluding itself)
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:100]

        # Create a set to keep track of unique book names
        unique_books = set()
        unique_recommendations = []

        for idx in [i[0] for i in sim_scores]:
            book_name = qualitative_data['Book-Title'].iloc[idx]
            if book_name not in unique_books and book_name != book_title:
                unique_books.add(book_name)
                unique_recommendations.append(book_name)

                if len(unique_recommendations) == num_recommendations:
                    break

        return unique_recommendations

In [8]:
print(get_unique_recommendations(book_title="Cracking the Coding Interview: 189 Programming Questions and Solutions", author="J.K. Rowling"))

['Great Answers to Tough Interview Questions', 'Your first interview: Everything you need to know to \\Ace\\" the interview process and get your first job"', 'Cracking the Gre 2002 (Cracking the Gre)', 'Helping Interview', 'Cracking the Love Code', 'Cracking the Sat 2003 (Cracking the Sat With Practice Tests on CD-Rom)', 'Cracking Open A Coffin', 'More Than a Carpenter', 'Interview with the Vampire : Anniversary edition', 'The Love Killer']


***Versione più efficente***

In [10]:
# Sample a smaller subset for development
sampled_data = data_qualitative.sample(frac=0.1, random_state=42)

# Combine text features into a single column
sampled_data['TextFeatures'] = sampled_data['Book-Title'] + ' ' + sampled_data['Book-Author']

# Create a TF-IDF vectorizer for text features (Author and Genre)
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Apply TF-IDF vectorization to text features
tfidf_matrix = tfidf_vectorizer.fit_transform(sampled_data['TextFeatures'])

# Convert tfidf_matrix to a sparse matrix
tfidf_sparse = tfidf_matrix.tocsr()

# Calculate cosine similarities between books based on text features
cosine_sim = linear_kernel(tfidf_sparse, tfidf_sparse)

# Function to recommend books based on content similarity with unique book names
def get_unique_recommendations(book_title, author, num_recommendations=10):
    global sampled_data

    if book_title not in sampled_data['Book-Title'].values:
        print(f"Warning: Book with title '{book_title}' not found in the dataset. Recommendations are based on available data.")
        # Add the new book to the dataset
        new_data = {'Book-Title': [book_title], 'Book-Author': [author]}
        sampled_data = sampled_data.append(pd.DataFrame(new_data), ignore_index=True)

        # Update TF-IDF matrix and cosine similarity matrix
        new_text_feature = f"{book_title} {author}"
        new_tfidf_vector = tfidf_vectorizer.transform([new_text_feature])
        tfidf_sparse.resize((tfidf_sparse.shape[0] + 1, tfidf_sparse.shape[1]))
        tfidf_sparse[-1] = new_tfidf_vector
        cosine_sim.resize((cosine_sim.shape[0] + 1, cosine_sim.shape[1] + 1))
        cosine_sim[-1] = linear_kernel(new_tfidf_vector, tfidf_sparse).flatten()

    idx = sampled_data[sampled_data['Book-Title'] == book_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort and get the top 100 most similar books (excluding itself)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:100]

    # Create a set to keep track of unique book names
    unique_books = set()
    unique_recommendations = []

    for idx in [i[0] for i in sim_scores]:
        book_name = sampled_data['Book-Title'].iloc[idx]
        if book_name not in unique_books and book_name != book_title:
            unique_books.add(book_name)
            unique_recommendations.append(book_name)

            if len(unique_recommendations) == num_recommendations:
                break

    return unique_recommendations

# Example usage
recommendations = get_unique_recommendations(book_title="Percy Jackson", author="Rick Riordan")
print(recommendations)

C:\Users\giaco\AppData\Local\Temp\ipykernel_39788\553892719.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_data = sampled_data.append(pd.DataFrame(new_data), ignore_index=True)
C:\Users\giaco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\sparse\_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


['Cold Springs', 'The Moviegoer', 'Cadre Lucifer', 'The Woman in the Moon and Other Tales of Forgotten Heroines', 'Oxford Book of Carols', "The Betrayer's Fortune: Dave &amp; Neta Jackson ; Text Illustrations by Julian Jackson (Trailblazer Books)", 'The concise Oxford dictionary of music,', "Rick Steves' Europe 101 (Europe 101: History and Art for the Traveler (Rick Steves))", "Rick Steves' Europe Through the Back Door 2001 (Rick Steves' Europe Through the Back Door, 19th ed)", 'Whispers']
